As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [1]:
import numpy as np
import pandas as pd

import os, time, math, json
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# api credentials
with open('/Users/hkim1297/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# instantiate YelpAPI variable
yelpApi = YelpAPI(login['api-key'], timeout_s=5.0)

In [5]:
# variables for query parameters
LOCATION = "Malibu, CA"
TERM = "Brunch"

In [18]:
# name of json file to be created
# NOTE: use different file name for each search query; include search terms in name
JSON_FILE = f"Data/results_in_progress_brunch.json"
JSON_FILE

'Data/results_in_progress_brunch.json'

In [19]:
# check if file already exists; if not, make it
fileExists = os.path.isfile(JSON_FILE)
if fileExists == False:
    # get folder name
    folder = os.path.dirname(JSON_FILE)
    # if it had a folder, make it
    if len(folder) > 0:
        os.makedirs(folder, exist_ok=True)
    # inform user and save empty list
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    # save first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([],f)
        
# if it already exists        
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_brunch.json not found. Saving empty list to file.


In [20]:
# load previous results and use amount as offset
with open(JSON_FILE, 'r') as f:
    prevResults = json.load(f)

# set offset
nResults = len(prevResults)
print(f"-{nResults} previous results found.")

-0 previous results found.


In [21]:
# make api call to find businesses
results = yelpApi.search_query(location=LOCATION,
                              term=TERM,
                              offset=nResults)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [22]:
# total results
results['total']

2800

In [23]:
# results per page
resultsPerPage = len(results['businesses'])
resultsPerPage

20

In [24]:
# calculate total # of pages
nPages = math.ceil((results['total'] - nResults)/resultsPerPage)
nPages

140

In [25]:
# add first page of results to file
prevResults.extend(results['businesses'])
with open(JSON_FILE, 'w') as f:
    json.dump(prevResults, f)

In [26]:
# for loop to call each page w/progress bar
for i in tqdm_notebook(range(1, 49)):
    time.sleep(.1) # adding 100ms pause in between loops
    # read in results to progress file
    with open(JSON_FILE, 'r') as f:
        prevResults = json.load(f)
    # save # of results as offset
    nResults = len(prevResults)
    results = yelpApi.search_query(location=LOCATION,
                                  term=TERM,
                                  offset=nResults)
    # add results to file and save
    prevResults.extend(results['businesses'])
    with open(JSON_FILE, 'w') as f:
        json.dump(prevResults, f)

  0%|          | 0/48 [00:00<?, ?it/s]

In [28]:
# convert json to df
df = pd.read_json(JSON_FILE)
display(df.head(), df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,IthDXFYZW1f5PRseK4sEVw,geoffreys-malibu-malibu,Geoffrey's Malibu,https://s3-media1.fl.yelpcdn.com/bphoto/cYCRFS...,False,https://www.yelp.com/biz/geoffreys-malibu-mali...,3094,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 34.025366, 'longitude': -118.769838}",[],$$$,"{'address1': '27400 Pacific Coast Hwy', 'addre...",+13104571519,(310) 457-1519,1699.051902
1,ih1OaYQh2kP3UTNHTpIHug,malibu-farm-pier-cafe-malibu,Malibu Farm Pier Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/3UPlD3...,False,https://www.yelp.com/biz/malibu-farm-pier-cafe...,1814,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.0,"{'latitude': 34.0356057756051, 'longitude': -1...",[],$$,"{'address1': '23000 Pacific Coast Hwy', 'addre...",+13104561112,(310) 456-1112,7226.446724
2,jutotRocXcPydWEcHALmHA,sparrow-cafe-malibu,Sparrow Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/ztpDOB...,False,https://www.yelp.com/biz/sparrow-cafe-malibu?a...,34,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 34.0385280804322, 'longitude': -1...",[],$$,"{'address1': '23847 Stuart Ranch Rd', 'address...",+13104563313,(310) 456-3313,5698.149673
3,G7BS2pFbis1KJTcHnIXI3g,the-sunset-restaurant-malibu,The Sunset Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/LJ2DK-...,False,https://www.yelp.com/biz/the-sunset-restaurant...,1495,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 34.01155, 'longitude': -118.817}",[pickup],$$,"{'address1': '6800 Westward Beach Rd', 'addres...",+13105891007,(310) 589-1007,6271.082781
4,tU1I5sBwyIY9ikF5kyCSOg,ollo-malibu,Ollo,https://s3-media1.fl.yelpcdn.com/bphoto/P8wJeP...,False,https://www.yelp.com/biz/ollo-malibu?adjust_cr...,850,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 34.03423676539962, 'longitude': -...",[],$$,"{'address1': '23755 Malibu Rd', 'address2': ''...",+13103171444,(310) 317-1444,5578.536633


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
975,_sla9w3jpglCnRQ7wqE_JA,funnel-mill-santa-monica-3,Funnel Mill,https://s3-media4.fl.yelpcdn.com/bphoto/B6SXwV...,False,https://www.yelp.com/biz/funnel-mill-santa-mon...,758,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 34.0194183, 'longitude': -118.488...","[pickup, delivery]",$$,"{'address1': '930 Broadway', 'address2': 'Ste ...",+13103931617,(310) 393-1617,24560.965989
976,mlpe75EZ88Lvf-vw7fckYg,cronies-sports-grill-agoura-hills,Cronies Sports Grill,https://s3-media3.fl.yelpcdn.com/bphoto/Tcjn87...,False,https://www.yelp.com/biz/cronies-sports-grill-...,246,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 34.15421, 'longitude': -118.75876}","[pickup, delivery]",$$,"{'address1': '5687 Kanan Rd', 'address2': '', ...",+18188890889,(818) 889-0889,13469.274948
977,50nZ4jdBYe6AuN0r3--KWg,turquoise-restaurant-redondo-beach,Turquoise Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/eX26Om...,False,https://www.yelp.com/biz/turquoise-restaurant-...,664,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 33.8173044046163, 'longitude': -1...",[pickup],$$,"{'address1': '1735 S Catalina Ave', 'address2'...",+13103733234,(310) 373-3234,41428.278838
978,xYdqVZtjXPv6M5Gd5pTQIQ,tom-tom-west-hollywood,Tom Tom,https://s3-media3.fl.yelpcdn.com/bphoto/deVMvX...,False,https://www.yelp.com/biz/tom-tom-west-hollywoo...,842,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.5,"{'latitude': 34.08407, 'longitude': -118.38491}",[delivery],$$$,"{'address1': '8932 Santa Monica Blvd', 'addres...",+13105438152,(310) 543-8152,34466.299476
979,_C-dloWW7NeSov8JPsh9mw,king-mediterrano-torrance-6,King Mediterrano,https://s3-media2.fl.yelpcdn.com/bphoto/2Gpkxs...,False,https://www.yelp.com/biz/king-mediterrano-torr...,1440,"[{'alias': 'mideastern', 'title': 'Middle East...",4.5,"{'latitude': 33.8741130815074, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '4354 Redondo Beach Blvd', 'addre...",+13103709438,(310) 370-9438,41123.305625


In [29]:
# check for dupes based on id#
df.duplicated(subset='id').sum()

0

In [30]:
# save as a .csv.gz
df.to_csv('Data/final_results_brunch.csv.gz',
         compression='gzip', index=False)